In [24]:
test = pd.DataFrame(columns=team_data.columns, data=np.zeros(len(team_data.columns)).reshape(1, len(team_data.columns)))

In [25]:
test["HomeTeam_Liverpool"] = 1
test["AwayTeam_Watford"] = 1

test["B365CH"] = 1.12
test["B365CD"] = 8
test["B365CA"] = 23
test["B365C>2.5"] = 1.33
test["B365C<2.5"] = 3.4

test = scaler.transform(test)

model.predict_proba(test)

array([[0.17636565, 0.82363435]])

In [26]:
1 / model.predict_proba(test)

array([[5.67003832, 1.21413101]])